In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [2]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train, x_test = x_train/255.0 *2-1 ,x_test/255.0 *2-1
print(x_train.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [3]:
N,H,W=x_train.shape
D=H*W
x_train=x_train.reshape(-1,D)
x_test=x_test.reshape(-1,D)
print(x_train.shape)

(60000, 784)


In [4]:
latent_dim=100 #

In [5]:
def build_gen(latent_dim):
  i=Input(shape=(latent_dim,))
  x=Dense(256,activation=LeakyReLU(alpha=0.2))(i)
  x=BatchNormalization(momentum=0.8)(x)
  x=Dense(512,activation=LeakyReLU(alpha=0.2))(i)
  x=BatchNormalization(momentum=0.8)(x)
  x=Dense(1024,activation=LeakyReLU(alpha=0.2))(i)
  x=BatchNormalization(momentum=0.8)(x)
  x=Dense(D,activation='tanh')(x)

  model=Model(i,x)
  return model


In [6]:
def build_des(img_size):
  i=Input(shape=(img_size,))
  x=Dense(512,activation=LeakyReLU(alpha=0.2))(i)
  x=Dense(256,activation=LeakyReLU(alpha=0.2))(x)
  x=Dense(1,activation='sigmoid')(x) #binary classification
  
  model=Model(i,x)
  return model
  

In [7]:
d=build_des(D) #28*28 = 784
d.compile(loss='binary_crossentropy',optimizer=Adam(0.0002,0.5),metrics=['accuracy'])

g=build_gen(latent_dim) #100

noise=Input(shape=(latent_dim,))
img=g(noise) #batch_size,784
d.trainable=False
fake_pred=d(img)

comb_model=Model(noise,fake_pred)

comb_model.compile(loss='binary_crossentropy',optimizer=Adam(0.0002,0.5))


In [8]:
#gen

batch_size=32
epochs=30000
sample_period=200 #every 200 steps gen and save some data 

#labels - real , fake
ones=np.ones(batch_size) #real
zeros=np.zeros(batch_size) #fake

d_losses=[]
g_losses=[]

if not os.path.exists('gan_imgs'):
  os.makedirs('gan_imgs')

In [9]:
def sample_imgs(epoch):
  rows,cols=5,5.

  noise=np.random.randn(rows*cols,latent_dim)
  
  imgs=g.predict(noise)

  #rescale 0-1
  imgs=0.5*imgs+0.5

  fig,axs =plt.subplots(rows,cols)
  idx=0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(H,W),cmap='gray')
      axs[i,j].axis('off')
      idx+=1
  fig.savefig('gan_imgs/%d.png' % epoch)
  plt.close

In [ ]:
#main training 

for epoch in range(epochs):

  #dis
  
  #real imgs
  idx=np.random.randint(0,x_train.shape[0],batch_size)
  real_imgs=x_train[idx]

  #fake imgs
  noise=np.random.randn(batch_size,latent_dim)
  fake_imgs=g.predict(noise)

  d_loss_real,d_acc_real=d.train_on_batch(real_imgs,ones)
  d_loss_fake,d_acc_fake=d.train_on_batch(fake_imgs,zeros)

  d_loss=0.5 *(d_loss_real+d_loss_fake)
  d_acc=0.5 *(d_acc_real+d_acc_fake)


  #gen
  noise=np.random.randn(batch_size,latent_dim)
  g_loss=comb_model.train_on_batch(noise,ones) #flip output label 

  d_losses.append(d_loss)
  g_losses.append(g_loss)

  if epoch%100==0:
    print(epoch+1,d_loss,d_acc,g_loss)
  
  if(epoch%sample_period == 0):
      sample_imgs(epoch)

                                          




1/1 [==============================] - 3s 3s/step
1 0.8131017088890076 0.21875 0.7053272724151611
1/1 [==============================] - 0s 13ms/step
101 0.00751256535295397 1.0 4.335890769958496
1/1 [==============================] - 0s 14ms/step
201 0.03756129456451163 1.0 6.161929130554199
1/1 [==============================] - 0s 13ms/step
301 0.2063743881881237 0.9375 4.941398620605469
1/1 [==============================] - 0s 19ms/step
401 0.5923863351345062 0.703125 1.0845415592193604
1/1 [==============================] - 0s 15ms/step
501 0.5676982700824738 0.703125 0.9557559490203857
1/1 [==============================] - 0s 13ms/step
601 0.6184936165809631 0.75 0.8589469194412231
1/1 [==============================] - 0s 16ms/step
701 0.5692017078399658 0.765625 0.9533029794692993
1/1 [==============================] - 0s 14ms/step
801 0.5754465609788895 0.78125 0.8672314882278442
1/1 [==============================] - 0s 16ms/step
901 0.5875215530395508 0.78125 0.88156193494

In [ ]:
plt.plot(g_losses,label='g_losses')
plt.plot(d_losses,label='d_losses')
plt.legend()

In [ ]:
from skimage.io imread
a=imread('gan_images/0.png')
plt.imshow(a)

In [ ]:
# # define the standalone discriminator model
# def define_discriminator(in_shape=(28,28,1)):
#  model = Sequential()
#  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
#  model.add(LeakyReLU(alpha=0.2))
#  model.add(Dropout(0.4))
#  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
#  model.add(LeakyReLU(alpha=0.2))
#  model.add(Dropout(0.4))
#  model.add(Flatten())
#  model.add(Dense(1, activation='sigmoid'))
#  # compile model
#  opt = Adam(lr=0.0002, beta_1=0.5)
#  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#  return model
 
# # define the standalone generator model
# def define_generator(latent_dim):
#  model = Sequential()
#  # foundation for 7x7 image
#  n_nodes = 128 * 7 * 7
#  model.add(Dense(n_nodes, input_dim=latent_dim))
#  model.add(LeakyReLU(alpha=0.2))
#  model.add(Reshape((7, 7, 128)))
#  # upsample to 14x14
#  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#  model.add(LeakyReLU(alpha=0.2))
#  # upsample to 28x28
#  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#  model.add(LeakyReLU(alpha=0.2))
#  model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
#  return model